In [1]:
import pandas as pd

import numpy as np
import nfl_data_py as nfl

In [2]:
dfNFL = nfl.import_pbp_data(years = list(range(1999, 2025, 1)), downcast = True, cache = False, alt_path = None)

1999 done.
2000 done.
2001 done.
2002 done.
2003 done.
2004 done.
2005 done.
2006 done.
2007 done.
2008 done.
2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
Downcasting floats.


In [3]:
# 14, 15, 16; Score, Final Score, Spread Final Score, Coaches, Quarterbacks, EPA, How good teams are passing
# running. Went for it on fourth down or for 2 earlier in the game. Time left in the game. 
# %of plays passes up till 4th quarter. Before and After. Timeouts remaining
# Variable Went for 2 got it, went for 2 didnt, didn't go for two got it, didn't go for 2 missed extra point

# Game Level Stats

In [4]:
list(dfNFL['play_type'].unique()) 

['kickoff',
 'pass',
 'run',
 'no_play',
 'punt',
 'extra_point',
 None,
 'field_goal',
 'qb_kneel',
 'qb_spike']

In [5]:
dfTemp = dfNFL[dfNFL['play_type'].isin(['pass', 'run', 'qb_kneel', 'qb_spike'])]

rates = (
    dfTemp
    .groupby(['game_id', 'posteam_type'])['qb_dropback']
    .mean()
    .unstack('posteam_type')      # columns: 'home', 'away'
    .rename(columns={'home': 'home_pass_rate',
                     'away': 'away_pass_rate'})
)

# merge back onto the original df by game_id
dfNFL = dfNFL.merge(rates, on='game_id', how='left')

In [6]:

dfNFL_filtered = dfNFL[
    (dfNFL['qtr'] == 4) &
    (dfNFL['score_differential'].isin([-16, -15, -14]))
]   # offense scores


In [7]:
selected_games = dfNFL_filtered['game_id'].unique()
dfNFL_selected = dfNFL[dfNFL['game_id'].isin(selected_games)]

In [8]:
columns_to_keep = ['play_id', 'game_id', 'home_team', 'away_team', 'season_type', 'week', 'posteam',
                  'posteam_type','defteam', 'side_of_field', 'yardline_100', 'game_date', 'quarter_seconds_remaining',
                   'half_seconds_remaining', 'game_seconds_remaining','game_half', 'quarter_end', 'drive', 
                   'sp', 'qtr', 'down', 'time', 'yrdln', 'desc','play_type', 'yards_gained', 'qb_dropback', 'air_yards',
                  'extra_point_result', 'two_point_conv_result', 'home_timeouts_remaining', 'away_timeouts_remaining',
                  'timeout', 'timeout_team', 'td_team', 'td_player_name', 'td_player_id', 'posteam_timeouts_remaining', 
                   'defteam_timeouts_remaining', 'total_home_score', 'total_away_score', 'posteam_score', 'defteam_score',
                  'score_differential', 'posteam_score_post', 'defteam_score_post', 'score_differential_post',
                  'ep', 'epa', 'total_home_epa', 'total_away_epa', 'total_home_rush_epa', 'total_away_rush_epa',
                  'total_home_pass_epa', 'total_away_pass_epa', 'air_epa', 'yac_epa', 'comp_air_epa', 'comp_yac_epa',
                  'total_home_comp_air_epa', 'total_away_comp_air_epa', 'total_home_comp_yac_epa', 'total_away_comp_yac_epa',
                  'total_home_raw_air_epa', 'total_away_raw_air_epa', 'total_home_raw_yac_epa', 'total_away_raw_yac_epa',
                  'first_down_rush', 'first_down_pass', 'third_down_converted', 'third_down_failed', 'fourth_down_converted',
                  'fourth_down_failed', 'incomplete_pass', 'interception','pass_attempt', 'touchdown',
                  'pass_touchdown', 'rush_touchdown', 'return_touchdown', 'extra_point_attempt', 'two_point_attempt',
                  'field_goal_attempt', 'fumble', 'complete_pass', 'passer_player_id', 'passer_player_name', 'passing_yards',
                  'receiver_player_id', 'receiver_player_name', 'rusher_player_id', 'rusher_player_name', 'rushing_yards',
                  'season', 'cp', 'cpoe', 'series', 'series_success', 'series_result', 'order_sequence', 'start_time', 'time_of_day',
                  'weather', 'stadium', 'play_clock', 'end_clock_time', 'end_yard_line', 'drive_play_count', 'drive_time_of_possession',
                  'drive_first_downs', 'drive_ended_with_score', 'drive_quarter_start', 'drive_quarter_end', 'drive_game_clock_start',
                  'drive_game_clock_end', 'drive_start_yard_line', 'drive_end_yard_line', 'total', 'spread_line', 'total_line',
                  'div_game', 'roof', 'temp', 'wind', 'home_coach', 'away_coach', 'stadium_id', 'game_stadium', 'aborted_play',
                  'success', 'passer', 'rusher', 'receiver', 'out_of_bounds', 'qb_epa', 'xyac_epa', 'xyac_mean_yardage', 
                  'xyac_median_yardage', 'xyac_success', 'xyac_fd', 'xpass', 'pass_oe', 'old_game_id_x', 'nflverse_game_id',
                  'possession_team', 'offense_formation', 'offense_personnel', 'defenders_in_box', 'defense_personnel',
                  'number_of_pass_rushers', 'time_to_throw', 'was_pressure', 'route', 'defense_man_zone_type', 'defense_coverage_type',
                  'offense_positions', 'defense_positions', 'away_pass_rate', 'home_pass_rate']

dfNFL_selected = dfNFL_selected[columns_to_keep]

In [9]:
qb_plays = dfNFL_selected[dfNFL_selected['qb_dropback'] == 1]

qb_map = (
    qb_plays[qb_plays['passer_player_name'].notna()]
    .groupby(['game_id', 'posteam_type'])['passer_player_name']
    .apply(lambda x: x.mode()[0])  # most common QB during dropbacks
    .unstack('posteam_type')       # creates only 'home' and 'away' columns
    .rename(columns={'home': 'home_qb', 'away': 'away_qb'})
)

dfNFL_selected = dfNFL_selected.merge(qb_map, on='game_id', how='left')

In [10]:
qb_plays = dfNFL_selected[dfNFL_selected['qb_dropback'] == 1]

# how many dropbacks each passer had for each game & team
qb_dropback_counts = (
    qb_plays
    .groupby(['game_id', 'posteam_type', 'passer_player_name'])
    .size()
    .reset_index(name='num_dropbacks')
)

min_dropbacks = 3  # tweak this as you like

qb_main = qb_dropback_counts[qb_dropback_counts['num_dropbacks'] >= min_dropbacks]

qb_counts = (
    qb_main
    .groupby(['game_id', 'posteam_type'])['passer_player_name']
    .nunique()
    .unstack('posteam_type')
    .fillna(0)
    .rename(columns={'home': 'home_qb_count', 'away': 'away_qb_count'})
)

qb_counts['home_qb_change'] = (qb_counts['home_qb_count'] > 1).astype(int)
qb_counts['away_qb_change'] = (qb_counts['away_qb_count'] > 1).astype(int)

dfNFL_selected = dfNFL_selected.merge(
    qb_counts[['home_qb_change', 'away_qb_change']],
    on='game_id',
    how='left'
)

In [11]:
rb_plays = dfNFL_selected[dfNFL_selected['play_type'] == 'run'].copy()

# 2. For each game + home/away, find the most common rusher (the main RB)
rb_map = (
    rb_plays[rb_plays['rusher_player_name'].notna()]
    .groupby(['game_id', 'posteam_type'])['rusher_player_name']
    .apply(lambda x: x.mode().iat[0])       # most common RB for that game/team
    .unstack('posteam_type')                # columns: 'home', 'away'
    .rename(columns={'home': 'home_rb', 'away': 'away_rb'})
)

# 3. Merge back to dfNFL_selected
dfNFL_selected = dfNFL_selected.merge(rb_map, on='game_id', how='left')

In [12]:
dfTemp = dfNFL_selected[dfNFL_selected['play_type'].isin(['pass', 'run', 'qb_kneel', 'qb_spike'])]

drive_rates = (
    dfTemp
    .groupby(['game_id', 'drive', 'posteam_type'])['qb_dropback']
    .mean()
    .unstack('posteam_type')   # columns: 'home', 'away'
    .rename(columns={
        'home': 'home_pass_rate_drive',
        'away': 'away_pass_rate_drive'
    })
    .reset_index()
)

dfNFL_selected = dfNFL_selected.merge(drive_rates, on=['game_id', 'drive'], how='left')

In [13]:
scoring_while_down = dfNFL_selected[
    (dfNFL_selected['qtr'] == 4) &
    (dfNFL_selected['score_differential'].isin([-14, -15, -16])) &      # offense is losing
    (dfNFL_selected['posteam_score_post'] > dfNFL_selected['posteam_score'])  # offense scores
].copy()

# 2️⃣ Identify the next drive number for those scoring drives
scoring_drives = (
    scoring_while_down[['game_id', 'drive']]
    .drop_duplicates()
)

scoring_drives['next_drive'] = scoring_drives['drive'] + 1

# 3️⃣ Pull ALL plays from those next drives into dfNFL_nextdrives
dfNFL_nextdrives = dfNFL_selected.merge(
    scoring_drives[['game_id', 'next_drive']],
    left_on=['game_id', 'drive'],
    right_on=['game_id', 'next_drive'],
    how='inner'
).drop(columns='next_drive')

# Optional: sort nicely
dfNFL_nextdrives = dfNFL_nextdrives.sort_values(['game_id', 'drive'])

In [14]:
prev_drive_plays = dfNFL_selected.merge(
    scoring_drives[['game_id', 'drive']],   # these are the scoring drives
    on=['game_id', 'drive'],
    how='inner'
)

pat_2pt_plays = prev_drive_plays[
    (prev_drive_plays['two_point_attempt'] == 1) |
    (prev_drive_plays['extra_point_attempt'] == 1)
].copy()

def classify_prev_drive(group):
    # 2-point attempt?
    if (group['two_point_attempt'] == 1).any():
        # any successful conversions?
        if ((group['two_point_attempt'] == 1) & (group['two_point_conv_result'] == 1)).any():
            return 'TD_2PT_good'
        else:
            return 'TD_2PT_failed'
    
    # extra point attempt?
    if (group['extra_point_attempt'] == 1).any():
        # any good XPs?
        if ((group['extra_point_attempt'] == 1) & (group['extra_point_result'] == 'good')).any():
            return 'TD_XP_good'
        else:
            return 'TD_XP_failed_or_blocked'
    
    # fallback if somehow no PAT/2pt is logged
    return 'TD_no_PAT_info'

drive_outcomes = (
    pat_2pt_plays
    .groupby(['game_id', 'drive'])
    .apply(classify_prev_drive)
    .reset_index(name='prev_drive_outcome')
)

scoring_drives = scoring_drives.merge(
    drive_outcomes,
    on=['game_id', 'drive'],
    how='left'
)

# now merge onto dfNFL_nextdrives using (game_id, drive == next_drive)
dfNFL_nextdrives = dfNFL_nextdrives.merge(
    scoring_drives[['game_id', 'next_drive', 'prev_drive_outcome']],
    left_on=['game_id', 'drive'],
    right_on=['game_id', 'next_drive'],
    how='left'
).drop(columns='next_drive')

In [15]:
# Get the down amount (14/15/16) for each scoring drive
down_map = (
    scoring_while_down
    .groupby(['game_id', 'drive'], as_index=False)['score_differential']
    .min()
)

down_map['down_amt'] = down_map['score_differential'].abs()

scoring_drives = scoring_drives.merge(
    down_map[['game_id', 'drive', 'down_amt']],
    on=['game_id', 'drive'],
    how='left'
)

import numpy as np

# Make sure outcome is present even if missing
scoring_drives['prev_drive_outcome'] = scoring_drives['prev_drive_outcome'].fillna('TD_no_PAT_info')

# helper booleans
is_14 = scoring_drives['down_amt'] == 14
is_15 = scoring_drives['down_amt'] == 15
is_16 = scoring_drives['down_amt'] == 16

is_2fail = scoring_drives['prev_drive_outcome'].eq('TD_2PT_failed')
is_xpgood = scoring_drives['prev_drive_outcome'].eq('TD_XP_good')
is_xpfail = scoring_drives['prev_drive_outcome'].eq('TD_XP_failed_or_blocked')

# 12 dummies
scoring_drives['d16_td_2pt_failed'] = (is_16 & is_2fail).astype(int)
scoring_drives['d16_td_xp_good']    = (is_16 & is_xpgood).astype(int)
scoring_drives['d16_td_xp_miss']    = (is_16 & is_xpfail).astype(int)

scoring_drives['d15_td_2pt_failed'] = (is_15 & is_2fail).astype(int)
scoring_drives['d15_td_xp_good']    = (is_15 & is_xpgood).astype(int)
scoring_drives['d15_td_xp_miss']    = (is_15 & is_xpfail).astype(int)

scoring_drives['d14_td_2pt_failed'] = (is_14 & is_2fail).astype(int)
scoring_drives['d14_td_xp_good']    = (is_14 & is_xpgood).astype(int)
scoring_drives['d14_td_xp_miss']    = (is_14 & is_xpfail).astype(int)

dummy_cols = [
    'd16_td_2pt_failed','d16_td_xp_good','d16_td_xp_miss',
    'd15_td_2pt_failed','d15_td_xp_good','d15_td_xp_miss',
    'd14_td_2pt_failed','d14_td_xp_good','d14_td_xp_miss',
]
is_2good = scoring_drives['prev_drive_outcome'].eq('TD_2PT_good')

scoring_drives['d16_td_2pt_good'] = (is_16 & is_2good).astype(int)
scoring_drives['d15_td_2pt_good'] = (is_15 & is_2good).astype(int)
scoring_drives['d14_td_2pt_good'] = (is_14 & is_2good).astype(int)

dummy_cols += ['d16_td_2pt_good','d15_td_2pt_good','d14_td_2pt_good']
dfNFL_nextdrives = dfNFL_nextdrives.merge(
    scoring_drives[['game_id', 'next_drive'] + dummy_cols],
    left_on=['game_id', 'drive'],
    right_on=['game_id', 'next_drive'],
    how='left'
).drop(columns='next_drive')

# Fill missing with 0 for drives that aren't in the "next drive after scoring" set
dfNFL_nextdrives[dummy_cols] = dfNFL_nextdrives[dummy_cols].fillna(0).astype(int)


In [16]:
dfNFL_kickoff = dfNFL_nextdrives[dfNFL_nextdrives['play_type'] == 'kickoff']

In [20]:
pd.set_option('display.max_columns', None)
dfNFL_kickoff.tail()

,play_id,game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,time,yrdln,desc,play_type,yards_gained,qb_dropback,air_yards,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,first_down_rush,first_down_pass,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,interception,pass_attempt,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,fumble,complete_pass,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,rusher_player_id,rusher_player_name,rushing_yards,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,weather,stadium,play_clock,end_clock_time,end_yard_line,drive_play_count,drive_time_of_possession,drive_first_downs,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,total,spread_line,total_line,div_game,roof,temp,wind,home_coach,away_coach,stadium_id,game_stadium,aborted_play,success,passer,rusher,receiver,out_of_bounds,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe,old_game_id_x,nflverse_game_id,possession_team,offense_formation,offense_personnel,defenders_in_box,defense_personnel,number_of_pass_rushers,time_to_throw,was_pressure,route,defense_man_zone_type,defense_coverage_type,offense_positions,defense_positions,away_pass_rate,home_pass_rate,away_qb,home_qb,home_qb_change,away_qb_change,away_rb,home_rb,away_pass_rate_drive,home_pass_rate_drive,prev_drive_outcome,d16_td_2pt_failed,d16_td_xp_good,d16_td_xp_miss,d15_td_2pt_failed,d15_td_xp_good,d15_td_xp_miss,d14_td_2pt_failed,d14_td_xp_good,d14_td_xp_miss,d16_td_2pt_good,d15_td_2pt_good,d14_td_2pt_good
8075,3393.0,2024_15_NE_ARI,ARI,NE,REG,15,ARI,home,NE,NE,35.0,2024-12-15,494.0,494.0,494.0,Half2,0.0,16.0,0.0,4.0,NaN,08:14,NE 35,13-J.Slye kicks 65 yards from NE 35 to end zon...,kickoff,0.0,0.0,NaN,None,None,3.0,2.0,0.0,None,None,None,None,3.0,2.0,23.0,10.0,23.0,10.0,13.0,23.0,10.0,13.0,1.487219,0.217665,15.899406,-15.899406,2.427979,-2.427979,7.895572,-7.895572,NaN,NaN,0.0,0.0,-5.475124,5.475124,10.817161,-10.817161,-3.215733,3.215733,7.938437,-7.938437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,None,NaN,None,None,None,None,NaN,2024,NaN,NaN,42.0,1.0,First down,3393.0,"12/15/24, 16:25:42",2024-12-15T23:51:31.823Z,"Sunny Temp: 68° F, Humidity: 18%, Wind: WSW 2 mph",State Farm Stadium,0,2024-12-15T23:51:35.007Z,None,9.0,4:47,4.0,1.0,4.0,4.0,08:14,03:27,ARI 30,NE 1,47,6.0,46.5,0,closed,NaN,NaN,Jonathan Gannon,Jerod Mayo,PHO00,State Farm Stadium,0.0,1.0,None,None,None,0.0,0.217665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024121507,2024_15_NE_ARI,ARI,None,"1 CB, 2 FS, 2 ILB, 1 OLB, 1 RB, 2 TE, 2 WR",0.0,"3 CB, 1 DE, 3 FS, 1 ILB, 1 K, 1 OLB, 1 TE",0.0,NaN,0.0,,,None,CB;FS;FS;ILB;ILB;OLB;RB;TE;TE;WR;WR,CB;CB;CB;DE;FS;FS;FS;ILB;K;OLB;TE,0.571429,0.562500,D.Maye,K.Murray,0,0,R.Stevenson,J.Conner,NaN,0.444444,TD_XP_good,0,0,0,0,0,0,0,1,0,0,0,0
8087,3863.0,2024_15_NE_ARI,ARI,NE,REG,15,ARI,home,NE,NE,35.0,2024-12-15,115.0,115.0,115.0,Half2,0.0,18.0,0.0,4.0,N

In [22]:
dfNFL_kickoff.shape

(1023, 180)